In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense, concatenate, Input, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow import keras

In [ ]:
from tensorflow.keras import backend as K
K.clear_session()

In [ ]:
# Load and preprocess the CSV data
df = pd.read_csv("transformed_df.csv")
def replace_with_unique(df, column: str):
    uniques = df[column].unique()
    mapping = dict(zip(uniques, range(0, len(uniques))))
    df[column].replace(mapping, inplace=True)
    return df
#replace_with_unique(df, "diagnostic")

In [ ]:
X = np.array(df.drop(["diagnostic", "img_id", "image_path"], axis=1))
y = np.array(df["diagnostic"])
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y)

In [ ]:
# Load and preprocess the images using ImageDataGenerator
image_directory = "./cancer/all_cancer_images/"
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 300
target_size = (224, 224)
class_mode = 'categorical'

train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=image_directory,
    x_col="img_id",
    y_col="diagnostic",
    target_size=target_size,
    batch_size=batch_size,
    class_mode=class_mode,
    shuffle=True,
)

xtrain, ytrain = next(train_generator)

In [ ]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.densenet import DenseNet121

# Define the CNN model
BASE_MODEL = ResNet50(
    weights='imagenet',
    include_top=False,
    pooling='avg',
    input_shape=(224, 224, 3)
)
BASE_MODEL.trainable = False

model_cnn = Sequential()
model_cnn.add(BASE_MODEL)
#model_cnn.add(keras.layers.Dense(units=2048, activation="relu"))
#model_cnn.add(keras.layers.Dense(units=512, activation="relu"))


# Define the DenseNet model
BASE_MODEL_DENSENET = DenseNet121(
    weights='imagenet',
    include_top=False,
    pooling='avg',
    input_shape=(224, 224, 3)
)
BASE_MODEL_DENSENET.trainable = False

model_densenet = Sequential()
model_densenet.add(BASE_MODEL_DENSENET)
#model_densenet.add(keras.layers.Dense(units=1024, activation="relu"))
#model_densenet.add(keras.layers.Dense(units=512, activation="relu"))

BASE_MODEL_MOBILENET = MobileNet(
    weights='imagenet',
    include_top=False,
    pooling='avg',
    input_shape=(224, 224, 3)
)
BASE_MODEL_MOBILENET.trainable = False
model_mobilenet = Sequential()
model_mobilenet.add(BASE_MODEL_MOBILENET)
#model_mobilenet.add(keras.layers.Dense(units=1024, activation="relu"))
#model_mobilenet.add(keras.layers.Dense(units=512, activation="relu"))

In [ ]:
# Define the ANN model
model_ann = Sequential()
input_ann = Input(shape=(X.shape[1]))
ann_output = Dense(units=15, activation='relu')(input_ann)
ann_output = Dense(units=8, activation='relu')(ann_output)
ann_output = Dense(units=4, activation='relu')(ann_output)

In [ ]:
def HyperModel(hyperparameters):
    '''
    creates a hypermodel by stacking dense layers on top of base model. 
    Two hyperparameters to be tuned: 
    1. number of neurons in the first dense layer,
    2. initial learning rate of the optimizer
    Args:
    hyperparameters - Keras tuner object
    '''
    # initialize the Sequential API to stack the layers
    uniques = df["diagnostic"].unique()
    model = keras.Sequential()
    
    # convolutional base 
    model.add(BASE_MODEL)
    
    # number of neurons in first dense layer
    hp_units = hyperparameters.Choice(
        'units', 
        values=[64,32,16,8]
    )
    # first dense layer
    model.add(
        keras.layers.Dense(
            units=hp_units, 
            activation='relu'
        )
    )
    # output layer with softmax activation function
    model.add(
        keras.layers.Dense(
            len(uniques),
            activation='softmax'
        )
    )
    # learning rate for the optimizer
    hp_learning_rate = hyperparameters.Choice(
        'learning_rate', 
        values=[1e-2, 1e-3, 1e-1]
    )
    # compile model
    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=hp_learning_rate
        ),
        loss=keras.losses.categorical_crossentropy,
        metrics=[keras.metrics.categorical_accuracy]
    )

    return model

In [ ]:
import keras_tuner as kt
# instantiate geridsearch
tuner = kt.GridSearch(
    hypermodel=HyperModel,
    objective='val_categorical_accuracy'
)

In [ ]:
# hypertuning settings summary
tuner.search_space_summary() 

In [ ]:
from keras.layers import concatenate
from tensorflow.keras.applications.efficientnet import EfficientNetB0


# Concatenate the models
concatenated = concatenate([model_cnn.output, model_densenet.output, model_mobilenet.output, ann_output])
#flat = Flatten()(concatenated)
ann_output_ = Dense(units=64, activation='relu')(concatenated)
ann_output_ = Dense(units=32, activation='relu')(ann_output_)
ann_output_ = Dense(units=16, activation='relu')(ann_output_)
ann_output_ = Dense(units=8, activation='relu')(ann_output_)
#ann_output_ = Dropout(rate=0.5)(ann_output_)
combined_output = Dense(units=4, activation='softmax')(ann_output_)

# Create the combined model
combined_model = Model(inputs=[model_cnn.input, model_densenet.input, model_mobilenet.input, input_ann], outputs=[combined_output])
combined_model.summary()


# Compile the model
from keras.optimizers import SGD
opt = SGD(lr=0.01)
combined_model.compile(optimizer=opt, loss=keras.losses.CategoricalCrossentropy(), metrics=[keras.metrics.CategoricalAccuracy()])

# Train the model
history = combined_model.fit([xtrain, xtrain, xtrain, X], ytrain, batch_size=32, epochs=1000, validation_split=0.2)
combined_model.save("combined_model.h5")